In [ ]:
import pandas as pd
import os
from statistics import mean
import matplotlib.pyplot as plot
import funkcije

starsevska_pot_plitka = os.getcwd()
starsevska_pot_globoka = os.path.join(starsevska_pot_plitka, 'podatkovna_baza')

Analizirali bomo naslednje:
- pri kateri starosti šahisti dosežejo svoj največji ELO (ter ali je ta podatek odvisen od tega kako dober je šahist)
- vpliv nacionalnosti to kdaj šahist doseže svoj največji ELO
- korelacija med ratingom (število točk ELO) in rankom (katero mesto zaseda šahist na lestvici najboljših 100)

Zanima nas pri kateri starosti šahisti dosežejo svoj vrhunec. Postopek, da pridobimo to informacijo bo naslednji:
1. Ustvarili bomo kopijo seznama sahisti ter v to kopijo shranili podatke o tem pri kateri starosti šahist doseže svoj največji ELO rating; podobno bomo naredili tudi z podatkom pri kateri starosti šahist doseže svoj največji indeks na lestvici najboljših 100 šahistov. Ta podatka nista nujno enakovredna, zaradi rating inflationa ([relevanten wikipedia snippet](https://en.wikipedia.org/wiki/Elo_rating_system#Mathematical_details)).
2. Statistično analizirali to - verjetno z preprosto aritmetično sredino.
3. Grafični prikaz podatkov.

In [ ]:
#Ker bomo analizirali starost pri kateri igralci dosežejo maksimalni rating v odvisnosti od države, moramo pobrati tudi podatke o slednjem.
#Pojavi se vprašanje kaj storiti, če šahist v svoji karieri igra za več kot eno državo. V okviru tega projekta bomo vse dosežke tega šahista upoštevali pod državo za katero je igral najprej.

stevec_drzav = {}
sahisti_elo = []

for filename in os.listdir(starsevska_pot_globoka):
    ime_sahista = filename.replace('_', ' ')
    trenutni_maksimum = [('01-2000', 0)]
    datotecna_pot = os.path.join(starsevska_pot_globoka, filename)
    podatki = pd.read_csv(datotecna_pot, index_col = 'datum', sep = ',')
    for datum, vrstica in podatki.iterrows():
        rating = int(vrstica['rating'])
        if rating > trenutni_maksimum[0][1]:
            trenutni_maksimum = [(str(datum), rating)]
        elif rating == trenutni_maksimum[0][1]:
            trenutni_maksimum.append((str(datum), rating))
    drzava = vrstica['drzava']
    if drzava in stevec_drzav.keys():
        stevec_drzav[drzava] += 1
    else:
        stevec_drzav[drzava] = 1
    sahisti_elo.append({'Ime': ime_sahista, 'Maksimumi' : trenutni_maksimum, 'Drzava' : vrstica['drzava']})
#sahisti_elo je seznam slovarjev, vsebujoc informacje o imenu, najvišjem ratnigu (oz pri kateri starosti ga je dosegel) ter državljanstvu 

print(sahisti_elo)
print(stevec_drzav)

#Sedaj bomo v seznam slovarjev sahisti_elo za vsakega šahista dodali še starost, pri kateri je dosegel maksimalni rating, če je teh več pa več starosti.
sahisti_csv_format = pd.read_csv(os.path.join(starsevska_pot_plitka, 'sahisti'), sep=',')

for sahist in sahisti_elo:
    ime_sahista = sahist['Ime']
    maksimumi = sahist['Maksimumi']
    sahist['Najvišji rating'] = maksimumi[0][1]
    sahist['Starosti ob najvišjih ratingih'] = []
    #Opomnik: Maksimumi je seznam nizov dveh podatkov, prvi je datum, drugi pa maksimalni elo šahista
    for indeks, vrstica in sahisti_csv_format.iterrows():
        if vrstica['Ime'] == ime_sahista:
            sahist['Leto rojstva'] = vrstica['Leto rojstva']
    for maksimum in maksimumi:
        sahist['Starosti ob najvišjih ratingih'].append(int(maksimum[0][3:]) - sahist['Leto rojstva'])
    sahist['Povprečna starost pri najvišjem ratingu'] = round(mean(sahist['Starosti ob najvišjih ratingih']), 1)
    sahist['Najmlajša starost pri najvišjem ratingu'] = min(sahist['Starosti ob najvišjih ratingih'])
    sahist['Največja starost pri najvišjem ratingu'] = max(sahist['Starosti ob najvišjih ratingih'])
sahisti_elo_pandas = pd.DataFrame(sahisti_elo, columns=['Ime', 'Leto rojstva','Drzava', 'Najvišji rating', 'Povprečna starost pri najvišjem ratingu', 'Najmlajša starost pri najvišjem ratingu', 'Največja starost pri najvišjem ratingu'])

sahisti_elo_pandas.head(250)

Uspešno smo sestavili razpredelnico, ki vsebuje imena vseh šahistov, leta rojstva, njihove najvišje ratinge ter povprečno starost pri najvišjem ratingu.
Zanima nas, pri kateri starosti šahisti v splošnem dosežejo svojo najvišji rating.

Da bi to analizirali uspešno moramo upoštevati še dejstvo, da nekateri igralci še niso dosegli svojega najvišjega ratinga; oz. ga trenutno dosegajo. Prvega faktorja žal ne moramo odpraviti, saj ne znamo predvideti, ali bo posameznik v prihodnosti postal boljši, slabši ali pa če bo celo nehal igrati šah na profesionalnem nivoju. Drugi faktor pa bomo nagovorili tako, da iz statistične analize eliminiramo vse igralce, ki so svoj najvišji rating dosegli v zadnjih 6 mesecih.

In [ ]:
shp = sahisti_elo_pandas
shp = shp[shp['Povprečna starost pri najvišjem ratingu'] - (2024 - shp['Leto rojstva']) <= 0.5]

for sahist in sahisti_elo:
    if sahist['Najvišji rating'] > 2850:
        print(sahist['Ime'], sahist['Maksimumi'])

print(f'Povprečna starost pri kateri šahist doseže svoj najvišji ELO rating je približno {round(mean(shp['Povprečna starost pri najvišjem ratingu']), 2)}. \nBralcu ob tej ugotovitvi puščam nekaj trenutkov za kontemplacijo o svoji smrtnosti ter življenskih dosežkih.')
#Bralcu ob tej ugotovitvi puščam nekaj trenutkov za kontemplacijo o svoji smrtnosti ter življenskih dosežkih.

plot.figure(figsize=(10,6))
plot.scatter(shp['Najvišji rating'], shp['Največja starost pri najvišjem ratingu'], label='Največja starost pri najvišjem ratingu v odvisnosti od ratinga', color='red', marker='.')
plot.scatter(shp['Najvišji rating'], shp['Najmlajša starost pri najvišjem ratingu'], label='Najmlajša starost pri najvišjem ratingu v odvisnosti od ratinga', color='blue', marker='.')
plot.scatter(shp['Najvišji rating'], shp['Povprečna starost pri najvišjem ratingu'], label='Povprečna starost pri najvišjem ratingu v odvisnosti od ratinga', color='purple', marker='.')



plot.title('Starost pri najvišjem ratingu v odvisnosti od najvišjega ratinga', fontsize = 14)
plot.ylabel('Najvišji rating', fontsize=12)
plot.xlabel('Starost', fontsize=12)

plot.show()

Ugotovili smo, da je med najboljšimi 100 šahisti na svetu povprečna starost, pri kateri dosežejo svoj največji ELO rating (kar verjetno sovpada z njihovo najvišjo sposobnostjo v igri šaha), priblžno 31.24. 

Diagram prikazuje starost pri kateri igralci dosežejo svoj najvišji rating. Vijolične točke predstavljajo povprečno starost posameznika, ko je dosegel svoj najvišji rating, modra in rdeča pa zaporedoma najnižjo in najvišjo starost ko je dosegel svoj najvišji rating.

Žal ne opazimo nobenega trenda v podatkih, je pa vredno omembe, da skoraj vsi igralci dosežejo svoj najvišji rating le enkrat v življenju, ter se na isti nivo ne povzpnejo ponovno. Izjema je Magnus Carlsen, ki je svoj najvišji rating 2882 ELO dosegel 05-2014 ter 08-2019. Da je Magnus Carlsen izjema, pa ni prav veliko presenečenje.

Za glavno jed bomo analizirali trende v ELO ratingu top 5 igralcev za obdobje zadnjih 24 let. Zakaj ravno top 5? Eden izmed razlogov je, da želimo te trende grafično prikazati, kar bi bilo težko za več kot 400 šahistov, za katere imamo podatke. 

To pomeni, da bomo za vsakega igralca, ki je v zadnjih 24 letih vstopil v top 5 igralcev, zabeležili ELO rating za vsak mesec, za katerega imamo podatke(sepravi za vsak mesec, ko je bil med najboljšimi 100 šahisti na svetu). Podatke bomo grafično predstavili kot zlomljenko.

In [25]:
#Napišimo program, ki zbere vse šahiste, ki so bili v top k-mestih; ter naloži njihove podake v slovar
# def podatki_top_nekaj(k):
#     podatki_sahistov_v_top_nekaj = {}
#     sahisti = pd.read_csv(os.path.join(starsevska_pot_plitka, 'sahisti'), index_col= 'Ime')
#     for ime, vrstica in sahisti.iterrows():
#         if sahist == 'Leto rojstva\'' or sahist == 'Leto rojstva':
#             pass
#         else:
#             podatki = pd.read_csv(os.path.join(starsevska_pot_globoka, ime.replace(' ', '_')))
#             for indeks, vrstica2 in podatki.iterrows():
#                 if vrstica2['rang'] <= 5:
#                     podatki_sahistov_v_top_nekaj[(ime, int(vrstica['Leto rojstva']))] = podatki['rang']
#     return podatki_sahistov_v_top_nekaj

def podatki_top_nekaj(k):
    sahisti_v_top_nekaj = set()
    sahisti = pd.read_csv(os.path.join(starsevska_pot_plitka, 'sahisti'), index_col= 'Ime')
    for ime, vrstica1 in sahisti.iterrows():
        if sahist == 'Leto rojstva\'' or sahist == 'Leto rojstva':
            pass
        else:
            podatki = pd.read_csv(os.path.join(starsevska_pot_globoka, ime.replace(' ', '_')))
            for indeks, vrstica2 in podatki.iterrows():
                if vrstica2['rank'] <= k:
                    sahisti_v_top_nekaj.add((ime, int(vrstica1['Leto rojstva'])))
    return sahisti_v_top_nekaj


#Vidimo, da je v zadnjih 24 letih 30 različnih šahistov zavzelo mesta med najboljšimi petimi.

#Narišimo sedaj še diagram
#Najprej naložimo podatke vsakega izmed šahistov:
# data_top_5 = {}
# for ime, leto_rojstva in podatki_top_nekaj(5):
#     data_top_5[(ime, leto_rojstva)] = pd.read_csv(os.path.join(starsevska_pot_globoka, ime.replace(' ', '_')))

#Sedaj podatke grafično predstavimo

plot.figure(figsize=(10,6))

for kljucni_nabor, podatki in podatki_top_nekaj(5).items():
    #Seznam podatkov o datumih nadomestimo z seznamom podatkov starosti posameznega sahista
    podatki_starost = [funkcije.datum_v_float(datum) - kljucni_nabor[1] for datum in podatki['datum']]
    print(podatki_starost)
    plot.plot(podatki_starost, podatki['rating'], label = kljucni_nabor[0])
plot.title('Rating v odvisnosti od starosti za šahiste med top 5 preteklih 24 let')
plot.legend(loc =  'upper left', bbox_to_anchor=(1,1))
plot.show()

KeyError: 'rank'

<Figure size 1000x600 with 0 Axes>

Diagram žal ni najbolj berljiv. Kako pa se obnaša povprečje teh šahistov pri vsaki starosti?

In [ ]:
#Ker nimamo podatkov o ratingu vsakeka izmed šahistov v podatki_top_nekaj(5) za vsako starost, bodisi ker starosti še niso dosegli, ker pri določeni starosti niso bili med najboljšimi 100 šahisti,
#bodisi ker so to starost dosegli pred letom 2000, bomo do podatkov o povprečju prišli na naslednji način.
povprecni_elo_pri_starosti_top_5 = {i: [] for i in range(15, 56)}
sahisti_v_top_5 = podatki_top_nekaj(5)
counter = 0
for sahist, leto_rojstva in sahisti_v_top_5:
    sahistov_record = pd.read_csv(os.path.join(starsevska_pot_globoka, sahist.replace(' ', '_')), index_col = 'datum')
    for datum, vrstica in sahistov_record.iterrows():
        leto = int(datum[3::])
        if leto - leto_rojstva >= 15 and leto - leto_rojstva <= 55:
            povprecni_elo_pri_starosti_top_5[leto - leto_rojstva].append(vrstica['rating'])

povprecje_2 = {i: [] for i in range(15, 56)}
for starost in povprecni_elo_pri_starosti_top_5:
    vsota = sum(povprecni_elo_pri_starosti_top_5[starost])
    dolzina = len(povprecni_elo_pri_starosti_top_5[starost])
    povprecje_2[starost] = round(vsota / dolzina, 2)

print(povprecje_2)

velikostni_ekstrem = []
najvisji = max(list(povprecje_2.values()))
for starost, rating_pri_starosti in povprecje_2.items():
    if rating_pri_starosti == najvisji:
        velikostni_ekstrem.append(starost)

print(velikostni_ekstrem)


pandas_thing = pd.Series(povprecje_2)

plot.figure(figsize=(10,6))

pandas_thing.plot(kind = 'line', marker = '.')

plot.xlabel('Starost')
plot.ylabel('Povprečni rating šahista, ki je dosegel top 5')
plot.show()

KeyError: 'rang'

In [ ]:
#Ker nimamo podatkov o ratingu vsakeka izmed šahistov v podatki_top_nekaj(5) za vsako starost, bodisi ker starosti še niso dosegli, ker pri določeni starosti niso bili med najboljšimi 100 šahisti,
#bodisi ker so to starost dosegli pred letom 2000, bomo do podatkov o povprečju prišli na naslednji način.
povprecni_elo_pri_starosti_top_5 = {i: [] for i in range(15, 56)}
sahisti_v_top_5 = podatki_top_nekaj(100)
counter = 0
for sahist, leto_rojstva in sahisti_v_top_5:
    sahistov_record = pd.read_csv(os.path.join(starsevska_pot_globoka, sahist.replace(' ', '_')), index_col = 'datum')
    for datum, vrstica in sahistov_record.iterrows():
        leto = int(datum[3::])
        if leto - leto_rojstva >= 15 and leto - leto_rojstva <= 55:
            povprecni_elo_pri_starosti_top_5[leto - leto_rojstva].append(vrstica['rating'])

povprecje_2 = {i: [] for i in range(15, 56)}
for starost in povprecni_elo_pri_starosti_top_5:
    vsota = sum(povprecni_elo_pri_starosti_top_5[starost])
    dolzina = len(povprecni_elo_pri_starosti_top_5[starost])
    povprecje_2[starost] = round(vsota / dolzina, 2)

print(povprecje_2)

velikostni_ekstrem = []
najvisji = max(list(povprecje_2.values()))
for starost, rating_pri_starosti in povprecje_2.items():
    if rating_pri_starosti == najvisji:
        velikostni_ekstrem.append(starost)

print(velikostni_ekstrem)


pandas_thing = pd.Series(povprecje_2)

plot.figure(figsize=(10,6))

pandas_thing.plot(kind = 'line', marker = '.')

plot.xlabel('Starost')
plot.ylabel('Povprečni rating šahista, ki je dosegel top 5')
plot.show()

KeyError: 'rang'

Opazimo podoben trend, kot smo ga videli pri

Kaj pa za k=1 ?

In [ ]:
data_top_1 = {}
for ime, leto_rojstva in podatki_top_nekaj(1):
    data_top_1[(ime, leto_rojstva)] = pd.read_csv(os.path.join(starsevska_pot_globoka, ime.replace(' ', '_')))

plot.figure(figsize=(10,6))

for kljucni_nabor, podatki in data_top_1.items():
    plot.plot(list(map(lambda x: datum_to_age_float(x, kljucni_nabor[1]), podatki['datum'])), podatki['rating'], label = kljucni_nabor[0])

plot.title('Rating v odvisnosti od starosti za najboljše šahiste preteklih 24 let')
plot.legend()
plot.show()